In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import numpy as np

# El Universal

In [2]:
def universal_scrap_links(url):
    
    links = []
    
    answer = requests.get(url).content        
    
    soup = BeautifulSoup(answer, 'html')        

    titles = soup.select('h3[class="ce3-AutorColumna_Titulo"]', href = True)
    
    for title in titles:
        links.append("https://www.eluniversal.com.mx"+title.select('a[href]')[0]['href'])

    titles = soup.select('[class="ce9-Tipo4_Titulo"]', href = True)
    
    for title in titles:
        links.append("https://www.eluniversal.com.mx"+title.select('a[href]')[0]['href'])
    return links

In [3]:
def universal_scrap_content(link):

    answer = requests.get(link).content

    answer = BeautifulSoup(answer)

    date = answer.select('[class="ce12-DatosArticulo_ElementoFecha"]')[0].text.strip()

    title = answer.select('[class="ceh-Opinion_Titulo"]')[0].text.strip()

    body = answer.select('[class="field field-name-body field-type-text-with-summary field-label-hidden"]')[0].text.strip()

    return title, date, body

In [4]:
def universal_scrap_columnista (url, auth_name):
    titles = []
    dates = []
    bodies = []
    links = []
    
    for link in universal_scrap_links(url):
        title, date, body = universal_scrap_content(link)
        titles.append(title)
        dates.append(date)
        bodies.append(body)
        links.append(link)
    data_dict = {'Title':titles, 'Date':dates, 'Body':bodies, 'Links':links}
    df = pd.DataFrame(data_dict)
    df['Author'] = auth_name
    df['Source'] = 'El Universal'
    return df[['Author', 'Title', 'Date', 'Body', 'Source','Links']]

# Milenio

In [5]:
global driver

def init_driver():
    global driver
    
    options = Options()
    options.add_argument("--headless")
    
    driver = webdriver.Chrome(executable_path='../../../Documents/chromedriver', options=options)
    driver.maximize_window()
    time.sleep(2)
    
    return driver

#driver.get("https://www.milenio.com/opinion/valeria-moy/page/1")

In [6]:
def milenio_url_scrapper(url_1):
    driver = init_driver()
    links = []
    
    driver.get(url_1)
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    try:
        driver.find_element_by_xpath('/html/body/div[4]/div/div/button').click()
    except:
        print('No cookies overlay detected.')
    
    while True:
        print(driver.current_url)
        respuesta = driver.page_source
        sopa = BeautifulSoup(respuesta, 'html')        
        titles = sopa.select('[class="title"]', href = True)
        
        for title in titles:            
            links.append('https://www.milenio.com' + title.select('a[href]')[0]['href'])
        try:
            if driver.current_url.split('/')[-1] == '1':
                driver.find_element_by_xpath('/html/body/div[1]/div/div/div[3]/div/div[2]/div[2]/a').click()
            else:
                driver.find_element_by_xpath('/html/body/div[1]/div/div/div[3]/div/div[2]/div[2]/a[2]').click()
            time.sleep(0.5)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        except Exception as e:
            print(e)
            break
        
    driver.quit()
    return links
    

In [7]:
def milenio_content_scrap(link):
    """"""
    respuesta = requests.get(link).content        
    sopa = BeautifulSoup(respuesta, 'html')      
    title = sopa.select('title')[0].text
    try:
        date = sopa.select('div time[class="date"]')[0].text
        body = sopa.select('div [class="media-container opinion"]', id = 'content-body')[0].text
    except Exception as e:
        #date = sopa.select('body > div.body-content > div > article > div.content-columns > div.column-right > div.nd-content-body > div.content-date > time')[0].text
        return '', '', ''
    
    return title, date, body

In [12]:
def milenio_scrap_columnista (url, auth_name):
    titles = []
    dates = []
    bodies = []
    links2 = []
    contador = 1
    
    links = milenio_url_scrapper(url)
    for link in links:
        title, date, body = milenio_content_scrap(link)
        print(f'Procesando: {contador} de {len(links)}')
        titles.append(title)
        dates.append(date)
        bodies.append(body)
        links2.append(link)
        contador += 1
    data_dict = {'Title':titles, 'Date':dates, 'Body':bodies, 'Links':links2}
    df = pd.DataFrame(data_dict)
    df['Author'] = auth_name
    df['Source'] = 'Milenio'
    return df

In [14]:
#uni_ric = universal_scrap_columnista('https://www.eluniversal.com.mx/autor-opinion/columnistas/ricardo-raphael/politica-zoom', 'Ricardo Raphael')
mil_ric = milenio_scrap_columnista('https://www.milenio.com/opinion/ricardo-rhapael/page/1', 'Ricardo Raphael')

https://www.milenio.com/opinion/ricardo-rhapael/page/1
https://www.milenio.com/opinion/ricardo-rhapael/page/2?
https://www.milenio.com/opinion/ricardo-rhapael/page/3?
https://www.milenio.com/opinion/ricardo-rhapael/page/4?
https://www.milenio.com/opinion/ricardo-rhapael/page/5?
https://www.milenio.com/opinion/ricardo-rhapael/page/6?
https://www.milenio.com/opinion/ricardo-rhapael/page/7?
https://www.milenio.com/opinion/ricardo-rhapael/page/8?
https://www.milenio.com/opinion/ricardo-rhapael/page/9?
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div/div/div[3]/div/div[2]/div[2]/a[2]"}
  (Session info: headless chrome=89.0.4389.82)

Procesando: 1 de 79
Procesando: 2 de 79
Procesando: 3 de 79
Procesando: 4 de 79
Procesando: 5 de 79
Procesando: 6 de 79
Procesando: 7 de 79
Procesando: 8 de 79
Procesando: 9 de 79
Procesando: 10 de 79
Procesando: 11 de 79
Procesando: 12 de 79
Procesando: 13 de 79
Procesando: 14 de 79
Procesando: 15 de 79
Pro

In [19]:
df_ric = pd.concat([uni_ric, mil_ric], axis = 0)
cols = [col.lower() for col in df_ric.columns]
cols
df_ric.columns = cols
df_ric.rename(columns={'links':'link'}, inplace = 1)
df_ric

,author,title,date,body,source,link
0,Ricardo Raphael,La reunión de la Conago: sin choques ni aspavi...,20/08/2020,Llegaron a la cita de San Luis Potosí en situa...,El Universal,https://www.eluniversal.com.mx/opinion/ricardo...
1,Ricardo Raphael,"A la hora de fabricar culpables, primero los p...",17/08/2020,Dylan Esaú y Adolfo Gómez son dos nombres cuyo...,El Universal,https://www.eluniversal.com.mx/opinion/ricardo...
2,Ricardo Raphael,Desigualdad y coronavirus en las Américas,06/08/2020,Los países más desiguales son los que están en...,El Universal,https://www.eluniversal.com.mx/opinion/ricardo...
3,Ricardo Raphael,Jiménez Espriú vs. López Obrador,23/07/2020,Hoy tendrá lugar la conversación pendiente ent...,El Universal,https://www.eluniversal.com.mx/opinion/ricardo...
4,Ricardo Raphael,El poder corruptor de Isabel Miranda,16/07/2020,Isabel Miranda Torres tiene experiencia en el ...,El Universal,https://www.eluniversal.com.mx/opinion/ricardo...
...,...,...,...,...,...,...
74,Ricardo Raphael,“Efecto corruptor” y consulta contra ex presid...,17.09.2020/01:16,\nArturo Zaldívar está emplazado a combatir “e...,Milenio,https://www.milenio.com/opinion/ricardo-raphae...
75,Ricardo Raphael,La rifa del avión presidencial fue un pésimo n...,14.09.2020/01:33,\nLa venta del avión presidencial explica cómo...,Milenio,https://www.milenio.com/opinion/ricardo-raphae...
76,Ricardo Raphael,El ogro y su impunidad,12.09.2020/01:01,\nSe derrumba el muro de la moral neoliberal y...,Milenio,https://www.milenio.com/opinion/ricardo-raphae...
77,Ricardo Raphael,Carlos Loret de Mola sube al estrado,10.09.2020/01:11,\nCarlos Loret de Mola Álvarez será citado a c...,Milenio,https://www.milenio.com/opinion/ricardo-raphae...


In [20]:
df_ric.to_csv('RicardoRaphael.csv')